# Gemini AI: Extracción avanzada de datos estructurados con Gemini Pro

Este notebook contiene un ejemplo práctico que demuestra cómo realizar búsquedas en Google en tiempo real y procesar los datos obtenidos mediante técnicas avanzadas de web scraping y técnicas avanzadas de extracción y procesamiento de información junto con la aplicación de modelos de inteligencia artificial como Gemini Pro de Vertex AI. Es una guía paso a paso que cubre desde la autenticación y configuración inicial hasta la extracción y formateo de datos.

**CONTENIDO**

- Autenticación en Google Cloud: Código para autenticar el usuario en Google Colab y configurar Vertex AI con un Project ID y una ubicación específica.
- Búsqueda en Tiempo Real en Google: Uso de herramientas como googlesearch para realizar búsquedas en Google y filtrar los resultados.
- Web Scraping con Langchain: Extracción de contenido web utilizando Langchain y conversión a texto con html2text.
- Extracción de Información: Procesos para extraer y estructurar información relevante del contenido web.
- Análisis de Datos: Técnicas para identificar temas, palabras clave, etiquetas y intenciones dentro del contenido extraído.


**CÓMO EMPEZAR**

- Autenticación y Configuración: Sigue las instrucciones para autenticarte y configurar tu entorno de Vertex AI.
- Extracción de Datos Avanzada: Realiza web scraping avanzado de una URL específica para recopilar datos.
- Análisis y Formateo de Datos con IA: Utiliza el modelo Gemini Pro para análisis avanzados y transformación de los datos en un formato estructurado y analizable.

**CONTRIBUIR**

Si tienes ideas, preguntas o deseas discutir sobre las posibilidades de la IA y cómo trabajar juntos para construir soluciones basadas en IAG, no dudes en contactarme:

- GitHub: https://github.com/albertgilopez
- LinkedIn: Albert Gil López: https://www.linkedin.com/in/albertgilopez/
- Inteligencia Artificial Generativa (IAG) en español: https://www.codigollm.es/

In [1]:
import sys

# Autenticación adicional para Google Colab
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

# @param {type:"string"}
PROJECT_ID = "vertex-ai-407803"  # Sustituye con tu Project ID
# @param {type:"string"}
LOCATION = "us-central1"  # Elige tu región

if "google.colab" in sys.modules:
    import vertexai
    vertexai.init(project=PROJECT_ID, location=LOCATION)

# import os
# print(os.environ)

import vertexai

from google.oauth2 import service_account
CREDENTIALS = service_account.Credentials.from_service_account_file(r"C:\Users\Usuario\Documents\personally.ai-shopper\Notebooks\vertex-ai-407803-ac70f5ed7cd1.json")

vertexai.init(project=PROJECT_ID, credentials=CREDENTIALS)


## Realtime Google Search with Langchain

En esta sección, vamos a demostrar cómo realizar búsquedas en tiempo real en Google utilizando Langchain, una biblioteca que facilita el trabajo con cadenas de lenguaje y procesamiento de texto. 

Nuestro objetivo es obtener resultados de búsqueda relevantes para una consulta dada y procesar estos resultados para su posterior análisis.


In [35]:
# Instalación de dependencias necesarias

!pip install -U duckduckgo_search googlesearch-python langchain playwright beautifulsoup4 html2text


Utilizaremos el módulo `googlesearch` para realizar búsquedas en Google. Definiremos una consulta de búsqueda y obtendremos los resultados.


In [41]:
google_search_results = []
structured_response = []

# Definición de la consulta de búsqueda
search_query = "juguetes en Cerdanyola del Vallès (Barcelona)"

# Realizando la búsqueda
from googlesearch import search
number_of_results = 1
results = search(search_query, lang="es", num_results=number_of_results)

# Filtrando y almacenando los resultados
google_search_results = [result for result in results if not result.startswith("https://www.tripadvisor.com")]

A continuación, procesaremos el primer enlace de los resultados de búsqueda para extraer información útil utilizando `langchain`.


In [42]:
# Procesamiento del primer enlace
link = google_search_results[0]
print("Procesando enlace:", link)

Procesando enlace: https://www.joguinesduba.com/


Utilizaremos funciones asíncronas para extraer y transformar el contenido HTML del enlace obtenido.


In [43]:
import html2text
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer

async def do_webscraping(link):
    try:
        urls = [link]
        loader = AsyncHtmlLoader(urls)
        docs = loader.load()

        html2text_transformer = Html2TextTransformer()
        docs_transformed = html2text_transformer.transform_documents(docs)

        if docs_transformed != None and len(docs_transformed) > 0:
            metadata = docs_transformed[0].metadata
            title = metadata.get('title', '')
            return {
                'summary': docs_transformed[0].page_content,
                'title': title,
                'metadata': metadata,
                'clean_content': html2text.html2text(docs_transformed[0].page_content)
            }
        else:
            return None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [44]:
response = await do_webscraping(link)

if response != None:
  structured_response.append(response)

Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.46s/it]


Visualizamos el resultado:

In [45]:
structured_response

[{'summary': 'Accedir\n\nRegistrar-me\n\n __\n\n\uf086\n\n\uf041\n\n\uf10b\n\n\\+ Duba Bodies\n\n\\+ Team Duba\n\nMi cuenta\n\n### Acceda a su cuenta\n\nUsuario\n\nContraseña\n\nRecordarme\n\n  * Olvido su password?\n  * Olvido su usuario?\n\n0\n\nSu pedido\n\nNo hay productos en su pedido.\n\n  * Inicio\n  * __ Juguetería\n  * __ Radiocontrol\n  * Servicios\n  * Team Duba\n  * Duba Bodies\n  * Contacto\n\nBuscar\n\n  *   * \n\n  * Radio control\n  * Juguetería\n\n  * ###  Tienda DUBA \n\nTenemos una **gran cantidad y variedad de artículos** para todo tipo de\nclientes: **coches de iniciación** y **competición** , sección de **juguetes**\n, **drones** , **helicópteros** , **coches de slot** y **marquetería**.\nSiempre intentamos asesorarte de la mejor manera, para que el producto que\nestá buscando se adapte a tus necesidades y características.\n\n    * Juguetería\n    * Radiocontrol\n\n  * ###  Servicios Radiocontrol \n\n**Todos los servicios** de apoyo que puedas necesitar en el hobb

## Information Extraction

En esta sección, nos enfocaremos en la extracción de información clave de los datos recopilados en la búsqueda anterior.

Utilizaremos técnicas de procesamiento de lenguaje natural para identificar y extraer información útil, como entidades, conceptos y relaciones, de los textos obtenidos.

Para estructurar mejor la información extraída, utilizaremos una función que extrae datos en formato JSON de nuestros textos procesados. Esto nos permitirá manejar la información de manera más eficiente en pasos posteriores.



In [46]:
import re
import json

def extract_json(input_string):
    # Extract JSON within {} block
    matches = re.findall(r'\{(.*?)\}', input_string, re.DOTALL)

    if matches:
        json_content = '{' + ''.join(matches) + '}'
        return json_content
    else:
        print("No JSON block found.")
        return None

Haremos uso de `vertexai` y su modelo generativo `gemini-pro` para procesar y extraer información relevante de los textos. Esto incluye resúmenes, palabras clave, y otros elementos significativos.

In [47]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def execute_prompt(prompt, max_output_tokens=8192):
  model = GenerativeModel("gemini-pro")
  responses = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": max_output_tokens,
        "temperature": 0,
        "top_p": 1
    },
  stream=True,
  )

  final_response = []

  for response in responses:
    if response.candidates and response.candidates[0].content.parts:
        final_response.append(response.candidates[0].content.parts[0].text)

  return ".".join(final_response)

Utilizaremos los textos procesados para generar prompts específicos, que luego serán utilizados con nuestro modelo generativo para extraer información relevante.

In [13]:
def get_text_extract_prompt(title, summary):
  prompt = f"""
  Here is its title: {title}
  Here is some text extracted:
  ---------
  {summary}
  ---------

  Web pages can have a lot of useless junk in them.
  For example, there might be a lot of ads, or a
  lot of navigation links, or a lot of text that
  is not relevant to the topic of the page. We want
  to extract only the useful information from the text.

  You can use the url and title to help you understand
  the context of the text.
  Please extract only the useful information from the text.
  Try not to rewrite the text, but instead extract
  only the useful information from the text.
  """
  return prompt

In [48]:
summarries = []

for structured_response_item in structured_response:
    title = structured_response_item['title']
    summary = structured_response_item['summary']
    if summary != "<html><body></body></html>":
      print(f'Summary for Title: {title}\n')
      text_extract_prompt = get_text_extract_prompt(title, summary)
      prompt_response = execute_prompt(text_extract_prompt)
      summarries.append(prompt_response)

Summary for Title: Duba - Tienda de radiocontrol y juguetes



Visualizamos el resultado:

In [49]:
summarries

['- DUBA is a toy and radio control store founded in 196.2.\n- They offer a wide variety of products, including initiation and competition. cars, toys, drones, helicopters, slot cars, and marquetry.\n- They provide professional maintenance, repair, and improvement services for radio control cars..\n- They have a large assortment of products and more than 65 square meters of spare parts to serve customers.\n- They specialize in advising and. teaching customers about technical concepts in the radio control hobby.']

## Topic Extraction

Esta sección se enfoca en la extracción de temas principales de los textos que hemos recopilado y procesado previamente.

Utilizaremos técnicas avanzadas de procesamiento de lenguaje natural para identificar los temas centrales y subtemas presentes en el contenido, lo que nos ayudará a comprender mejor la naturaleza de los datos.

Para extraer temas de manera efectiva, generaremos prompts específicos que guiarán a nuestro modelo generativo en la identificación de los temas principales y subtemas en el texto.

In [51]:
def get_topic_extraction_prompt(content):
    prompt = f"""Label the main topic or topics in the following text: {content}"""
    prompt = prompt + """1. Identify and list the primary topic or category or provide a short description of the main subject matter of the text.
      2. If there are subtopics or secondary themes mentioned in the text, list them as well. If the text discusses multiple topics, provide a list of these topics and describe their relevance.
      3. Consider the context and tone of the text to determine the most appropriate topics. Take into account keywords, phrases, or specific terms that relate to the topics.
      4. If any notable entities (people, places, brands, products, etc.) are mentioned in the text that play a role in the topics, mention them and their associations.
      5. If the text suggests any actions, decisions, or recommendations related to the identified topics, provide a brief summary of these insights.

      Ensure that your labeling is clear, concise, and reflects the most significant topics or categories found in the text.

      Here's the output schema:

      ```
      {
          "Topic": "",
          "Subtopics": [""],
          "Context": "",
          "NotableEntities": [],
          "Recommendations": ""
      }
      ```

      Do not respond with your own suggestions or recommendations or feedback."""
    return prompt

Haremos uso del modelos generativo `gemini-pro` de `vertexai`, para procesar los prompts y extraer los temas relevantes del texto.

In [52]:
topics = []

for structured_response_item in structured_response:
  title = structured_response_item['title']
  summary = structured_response_item['summary']
  if summary != "<html><body></body></html>":
    print(f'Topics: {title}\n')
    prompt = get_topic_extraction_prompt(summary)
    response = execute_prompt(prompt)
    print(response)
    


Topics: Duba - Tienda de radiocontrol y juguetes

",
  "Subtopics": [
    "Juguetería",.
    "Modelismo",
    "Radiocontrol",
    "Slot",
    "Competición",
    "Servicios de mantenimiento, reparación y. mejora de coches de radiocontrol"
  ],
  "Context": "La empresa Duba es una tienda especializada en juguetes, modelismo, radiocontrol., slot y competición desde 1962. Ofrecen una gran variedad de artículos para todo tipo de clientes, desde coches de iniciación hasta coches de competición, así como una sección de juguetes, drones, helicópteros, coches de. slot y marquetería.",
  "NotableEntities": [
    "Duba",
    "Joaquim Dubà"
  ],
  "Recommendations": "La empresa Duba recomienda a sus clientes que se ases.oren con ellos para elegir el producto que mejor se adapte a sus necesidades y características."
}
```


Añadimos un paso extra, un nuevo prompt para validar la relevancia y la calidad del contenido proporcionado.

Aprovechamos para solicitarle mejoras y acciones de mejora asociadas.

In [53]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def validate_data_with_gemini(content):
    """
    Esta función utiliza el modelo Gemini para validar la relevancia y calidad del contenido proporcionado.
    Devuelve un texto que indica si el contenido es relevante y de calidad para el procesamiento posterior.

    Args:
        content (str): El contenido a validar.

    Returns:
        str: Respuesta del modelo indicando la relevancia y calidad del contenido.
    """

    # Configura el modelo Gemini
    model = GenerativeModel("gemini-pro")

    # Crea el prompt para el modelo
    prompt = f"""
    You are a data validation expert. Your job is to analyze the following content and determine if it is relevant and of good quality for further processing. Here is the content:
    ---
    {content}
    ---

    Please provide your assessment on the quality and relevance of this content for further information extraction processes.
    Please provide improvement possibilities if there is possible.
    Please provide actions to do according with this improvement possibilities.
    """

    # Ejecuta el modelo con el prompt
    response = model.generate_content(
        prompt,
        generation_config={
            "max_output_tokens" : 8192,
            "temperature": 0,
            "top_p": 1
        },
    )

    # Obtiene y retorna la respuesta del modelo
    if response.candidates and response.candidates[0].content.parts:
        return response.candidates[0].content.parts[0].text
    else:
        return "No se pudo obtener una respuesta válida del modelo."

# Ejemplo de uso
content_to_validate = response
validation_result = validate_data_with_gemini(content_to_validate)
print(validation_result)


**Assessment:**

The content provided is relevant to the topic of toys, modeling, radio control, slot cars, and competition. It also mentions the company Duba, which specializes in these areas. The content is of good quality, as it is well-written and provides a clear overview of the company and its products.

**Improvement Possibilities:**

* The content could be improved by adding more specific information about the products that Duba offers. For example, it could include a list of the different types of toys, models, and radio-controlled cars that the company sells.
* The content could also be improved by adding more information about the company's history and its founder, Joaquim Dubà. This would help to provide a more personal touch to the content and make it more engaging for readers.

**Actions to Do:**

* To improve the content, the following actions could be taken:
    * Add a list of the different types of toys, models, and radio-controlled cars that Duba sells.
    * Add mor

## Keyword Extraction

En esta sección, abordaremos el proceso de identificar y extraer las palabras clave más relevantes de los textos procesados anteriormente.

Esta técnica es crucial para entender rápidamente los temas centrales del texto y puede ser de gran utilidad en aplicaciones como SEO, resumen de contenido y análisis temático.

Para extraer palabras clave de manera eficiente, formularemos prompts específicos que dirigirán nuestro modelo generativo a enfocarse en las palabras y frases más significativas del contenido.


In [23]:
def get_keyword_extraction_prompt(content):
    prompt = f"""Extract key keywords or phrases from the following text: {content}"""
    prompt = prompt + """1. Identify and list the most important keywords or key phrases in the text. These keywords should capture the main topics, concepts, or subjects discussed in the text.
      2. If there are subtopics or secondary themes mentioned in the text, list them as well. Ensure that the extracted keywords accurately represent the content's context.
      3. Include the exact text span or sentence where each keyword or phrase is found in the original text.
      4. If there are any ambiguous keywords or phrases, indicate the uncertainty and provide possible interpretations or context that might clarify the intended meaning.
      5. Consider the context, relevance, and frequency of the keywords when determining their significance.
      6. If the text suggests any actions, decisions, or recommendations related to the extracted keywords, provide a brief summary of these insights.

      Ensure that your keyword extraction results are relevant, concise, and capture the essential topics within the text.

      Here's the output schema:

      ```
      {
          "KeywordExtraction": [
              {
                  "Keyword": "",
                  "Context": "",
                  "TextSpan": ""
              }
          ]
      }
      ```

      Do not respond with your own suggestions or recommendations or feedback.
    """
    return prompt

Utilizaremos el modelo generativo `gemini-pro` proporcionado por `vertexai` para procesar los prompts creados y extraer las palabras clave relevantes.

In [54]:
keywords = []

for summary in summarries:
  if summary != "":
    prompt = get_keyword_extraction_prompt(summary)
    response = execute_prompt(prompt)
    extracted_json = extract_json(response)
    if extracted_json != None:
      keywords.append(extracted_json)

Visualizamos el resultado:

In [55]:
keywords

['{\n  "KeywordExtraction": [\n    {\n      ".Keyword": "DUBA",\n      "Context": "DUBA is a. toy and radio control store founded in 196.2.",\n      "TextSpan": "DUBA is a toy and radio control store founded in .196.2."\n    \n      "Keyword": "Toys",\n      "Context": "They offer a wide variety of. products, including initiation and competition. cars, toys, drones, helicopters, slot cars, and marquetry.",\n      "TextSpan": "They offer a wide variety of products, including initiation and competition. cars, toys, drones,. helicopters, slot cars, and marquetry."\n    \n      "Keyword": "Radio Control Cars",\n      "Context": "They provide professional maintenance, repair, and improvement services for radio control cars.",\n.      "TextSpan": "They provide professional maintenance, repair, and improvement services for radio control cars."\n    \n      "Keyword": "Spare Parts",\n      "Context": "They have a large assortment of products and more than 65 square meters of spare parts to ser

## Automated Tagger

En esta sección, nos centraremos en el etiquetado automático de los textos. 

El etiquetado automático implica identificar y asignar etiquetas o categorías relevantes a segmentos de texto, lo que facilita su clasificación y análisis posterior.

Esta técnica es especialmente útil para organizar grandes volúmenes de datos y extraer insights significativos.


In [56]:
def get_automated_tagger_extraction_prompt(content):
    prompt = f"""Automate the tagging of the following unstructured data: {content}"""
    prompt = prompt + """1. Identify and extract the most relevant tags, keywords, or categories for the given data. These tags should succinctly represent the content's main themes, subjects, or topics.
        2. List the extracted tags, and provide a brief description or rationale for each tag to help users understand their significance.
        3. If there are subcategories or hierarchies in the tags, ensure that they are appropriately nested or organized.
        4. Consider the context, content, and domain-specific knowledge when selecting tags. Ensure that the tags accurately reflect the essence of the data.
        5. If any tags are ambiguous or could have multiple interpretations, address these challenges and provide explanations for the chosen tags.
        6. If there are specific tasks or analyses where the tagged data will be used, describe these use cases and how the tags are expected to be applied.
        7. If the data contains temporal or dynamic elements, mention any trends, changes, or time-sensitive aspects that might impact the tags.

        Ensure that your automated tagging results are clear, relevant, and make the data more accessible and useful.

        Here's the output schema:

        ```
        {
            "AutomatedTagging": {
                "Tags": [
                    {
                        "Tag": "",
                        "Sentences": []
                    }
                ]
            }
        }
        ```

        Do not respond with your own suggestions or recommendations or feedback.
    """
    return prompt

Utilizaremos el modelo generativo `gemini-pro` de `vertexai` para procesar los prompts y realizar el etiquetado automático de los textos, lo que nos ayudará a clasificar y entender mejor el contenido.

In [57]:
taggers = []

for summary in summarries:
  if summary != "":
    prompt = get_automated_tagger_extraction_prompt(summary)
    response = execute_prompt(prompt)
    extracted_json = extract_json(response)
    if extracted_json != None:
      taggers.append(extracted_json)

Visualizamos el resultado:

In [58]:
taggers

['{\n    "AutomatedTagging": {\n        "Tags":. [\n            {\n                "Tag": "Radio Control Store",\n                ."Sentences": [\n                    "DUBA is a toy and radio control store founded in 196.2."\n                ]\n            \n                "Tag": "Products",\n                "Sentences": [\n                    "They offer a wide variety of products, including initiation and. competition. cars, toys, drones, helicopters, slot cars, and marquetry."\n                ]\n            \n                "Tag": "Services",\n                "Sentences": [\n                    "They provide professional. maintenance, repair, and improvement services for radio control cars."\n                ]\n            \n                "Tag": "Spare Parts",\n                "Sentences": [\n                    "They have a large assortment of products and. more than 65 square meters of spare parts to serve customers."\n                ]\n            \n                "Tag": "

## Intent Extraction

En esta sección, nos enfocaremos en la extracción de intenciones (intents) de los textos procesados.

La extracción de intenciones es crucial en aplicaciones como el desarrollo de chatbots y asistentes virtuales, ya que nos permite entender mejor lo que el usuario desea o necesita a partir de su entrada de texto.

Formularemos prompts que guiarán a nuestro modelo generativo a identificar las intenciones subyacentes en el texto, lo cual es un paso esencial para comprender las necesidades y deseos expresados en el contenido.



In [31]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def get_intent_extraction_prompt(content):
    # Definir el esquema en formato JSON correctamente
    schema = """
    {
        "intents": [
            {
                "intent": "",
                "statement": ""
            }
        ]
    }
    """
    prompt = f"""You are an expert intent detector. Your job is to detect and list down all the intents within the below content. Output the same in the specified JSON schema format.
    Here's the content:
    ---
    {content}
    ---
    Here's the schema:
    {schema}
    Do not respond with your own suggestions or recommendations or feedback.
    """
    return prompt

Utilizaremos el modelo generativo `gemini-pro` de `vertexai` para procesar los prompts y detectar las intenciones presentes en los textos. Esto nos ayudará a analizar y categorizar las entradas de texto de manera efectiva.

In [59]:
intents = []

# Crear y ejecutar el prompt
instruct_prompt = "get me the summary for the following content"
prompt = get_intent_extraction_prompt(instruct_prompt)
response = execute_prompt(prompt)
extracted_json = extract_json(response)

# Añadir el JSON extraído si existe
if extracted_json:
    intents.append(extracted_json)

Visualizamos el resultado:

In [60]:
print(intents)

['{\n  "intents": [\n    {\n      ."intent": "GetSummary",\n      "statement": "get me the. summary for the following content"\n    }']
